# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/joseignacio/Desktop/python-api-challenge/WeatherPy/Resources/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,9.43,54,5,2.93,GS,1738079249
1,1,yellowknife,62.4560,-114.3525,-12.71,85,100,3.60,CA,1738079272
2,2,edinburgh of the seven seas,-37.0676,-12.3116,16.22,75,100,11.72,SH,1738079273
3,3,pontes e lacerda,-15.2261,-59.3353,34.67,43,20,2.07,BR,1738079277
4,4,geraldton,-28.7667,114.6000,25.23,78,89,2.06,AU,1738079278


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [88]:
%%capture --no-display


# Configure the map plot
city_data_df = city_data_df.rename(columns={'Lat': 'latitude', 'Lng': 'longitude'})


city_map = city_data_df.hvplot.points(
    'longitude', 
    'latitude', 
    geo=True,                     
    tiles='OSM',                  
    size=100,                      
    color='City',                
    legend='right',               
    frame_width=800,             
    frame_height=500,            
    alpha=0.7,                    
    hover_cols=['City', 'latitude', 'longitude', 'Humidity'],  
    line_width=0                 
)

# Display the map
city_map



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [68]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 33) & 
    (city_data_df['Wind Speed'] < 9.5) & 
    (city_data_df['Cloudiness'] == 0)
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,latitude,longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,viedma,-40.8135,-62.9967,30.51,14,0,5.23,AR,1738079279
42,42,alaghsas,17.0187,8.0168,29.94,11,0,8.23,NE,1738079320
51,51,faya-largeau,17.9257,19.1043,27.94,13,0,8.23,TD,1738079330
85,85,nouadhibou,20.9310,-17.0347,30.99,22,0,5.66,MR,1738079369
111,111,pisco,-13.7000,-76.2167,24.03,78,0,2.06,PE,1738079335
127,127,mar del plata,-38.0023,-57.5575,22.19,67,0,5.66,AR,1738079137


### Step 3: Create a new DataFrame called `hotel_df`.

In [76]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'latitude', 'longitude', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,latitude,longitude,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,54,
1,yellowknife,CA,62.4560,-114.3525,85,
2,edinburgh of the seven seas,SH,-37.0676,-12.3116,75,
3,pontes e lacerda,BR,-15.2261,-59.3353,43,
4,geraldton,AU,-28.7667,114.6000,78,
...,...,...,...,...,...,...
129,kloulklubed,PW,7.0419,134.2556,76,
130,antofagasta,CL,-23.6500,-70.4000,67,
131,stanley,GB,54.8680,-1.6985,94,
132,kununurra,AU,-15.7667,128.7333,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [82]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['latitude']
    longitude = row['longitude']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
pontes e lacerda - nearest hotel: Hotel Avenida
geraldton - nearest hotel: Ocean Centre Hotel
viedma - nearest hotel: Níjar
la mira - nearest hotel: Hotel La Loma
ta`u - nearest hotel: No hotel found
amga - nearest hotel: "АммА"
thompson - nearest hotel: Thompson Inn
waitangi - nearest hotel: Hotel Chathams
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
tevriz - nearest hotel: No hotel found
altamont - nearest hotel: Quality Inn
kavaratti - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
liski - nearest hotel: Гостиница
xam nua - nearest hotel: Phetmeuangsam Hotel
maturin - nearest hotel: Hotel Rossi
puerto natales - nearest hotel: Dorotea Patagonia Hostel
galliano - nearest hotel: Fairfield Inn & Suites Cut Off-Galliano
danli - nearest h

,City,Country,latitude,longitude,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,54,No hotel found
1,yellowknife,CA,62.4560,-114.3525,85,No hotel found
2,edinburgh of the seven seas,SH,-37.0676,-12.3116,75,No hotel found
3,pontes e lacerda,BR,-15.2261,-59.3353,43,Hotel Avenida
4,geraldton,AU,-28.7667,114.6000,78,Ocean Centre Hotel
...,...,...,...,...,...,...
129,kloulklubed,PW,7.0419,134.2556,76,The Adventures Inn
130,antofagasta,CL,-23.6500,-70.4000,67,Hotel Colón
131,stanley,GB,54.8680,-1.6985,94,Hotel 52
132,kununurra,AU,-15.7667,128.7333,89,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [130]:
# Filter out rows where 'Hotel Name' is 'Hotel not found'
hotel_df_filtered = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Verify the filtering
hotel_df_filtered

,City,Country,latitude,longitude,Humidity,Hotel Name
3,pontes e lacerda,BR,-15.2261,-59.3353,43,Hotel Avenida
4,geraldton,AU,-28.7667,114.6000,78,Ocean Centre Hotel
5,viedma,AR,-40.8135,-62.9967,14,Níjar
6,la mira,MX,18.0350,-102.3266,71,Hotel La Loma
8,amga,RU,60.8953,131.9608,99,"""АммА"""
...,...,...,...,...,...,...
128,ankang,CN,32.6800,109.0172,53,金友商务酒店
129,kloulklubed,PW,7.0419,134.2556,76,The Adventures Inn
130,antofagasta,CL,-23.6500,-70.4000,67,Hotel Colón
131,stanley,GB,54.8680,-1.6985,94,Hotel 52


In [136]:
# %%capture --no-display

# Configure the map plot

hotel_map = hotel_df_filtered.hvplot.points(
    x='longitude', 
    y='latitude', 
    geo=True,                     
    tiles='OSM',                  
    size=150,                     
    color='City',                  
    legend='right',                
    frame_width=1000,              
    frame_height=550,              
    alpha=0.9,                     
    hover_cols=['City', 'Country', 'Hotel Name', 'latitude', 'longitude', 'Humidity'],  
    line_width=0,                  
    marker='circle'                
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (City,Country,Hotel Name,Humidity)